In [10]:
from lxml import html
import requests
import re
import pandas as pd

# SCRAPING

In [5]:
def process(season, episode, stripped):
    filtered = [s for s in stripped if not s.startswith("/") and s != "More on Genius"]
    labeled = []
    for f in filtered:
        m = re.search('([A-Z ]+):(.*)', f)
        if m:
            labeled.append([m.group(1),m.group(2)])
        elif f.startswith("["):
            labeled.append(["SCENE", f])
        else:
            labeled.append([f])
    merged = []
    chunks = []
    cur = "DEFAULT"
    for l in labeled:
        if len(l) >1:
            merged.append([cur,chunks])
            chunks = []
            cur = l[0]
            chunks.append(l[1])
        else:
            chunks.append(l[0])

    return [[season, episode, m[0], " ".join(m[1]).strip()] for m in merged]



In [8]:
# these were in bad shape:
#  "https://genius.com/albums/Game-of-thrones/Season-7-scripts"


seasons = [
    "https://genius.com/albums/Game-of-thrones/Season-1-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-2-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-3-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-4-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-5-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-6-scripts"
]

episodes = [];
for sidx, url in enumerate(seasons):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    ep_urls = tree.xpath("//div[contains(@class, 'chart_row')]/a/@href")    
    for eidx, ep in enumerate(ep_urls):
        if "needs-editing" in ep:
            print("skipping: ", ep)
            continue
        print(sidx, eidx, ep)
        page = requests.get(ep)
        tree = html.fromstring(page.content)
        raw = tree.xpath("//div[contains(@class, 'lyrics')]/descendant::*/text()")
        stripped = [r.strip() for r in raw if r.strip() != '']
        processed = process(sidx, eidx, stripped)
        episodes = episodes + processed
    

0 0 https://genius.com/Game-of-thrones-winter-is-coming-annotated
0 1 https://genius.com/Game-of-thrones-the-kingsroad-annotated
0 2 https://genius.com/Game-of-thrones-lord-snow-annotated
0 3 https://genius.com/Game-of-thrones-cripples-bastards-and-broken-things-annotated
0 4 https://genius.com/Game-of-thrones-the-wolf-and-the-lion-annotated
0 5 https://genius.com/Game-of-thrones-a-golden-crown-annotated
0 6 https://genius.com/Game-of-thrones-you-win-or-you-die-annotated
0 7 https://genius.com/Game-of-thrones-the-pointy-end-annotated
0 8 https://genius.com/Game-of-thrones-baelor-annotated
0 9 https://genius.com/Game-of-thrones-fire-and-blood-annotated
1 0 https://genius.com/Game-of-thrones-the-north-remembers-annotated
1 1 https://genius.com/Game-of-thrones-the-night-lands-annotated
1 2 https://genius.com/Game-of-thrones-what-is-dead-may-never-die-annotated
1 3 https://genius.com/Game-of-thrones-garden-of-bones-annotated
1 4 https://genius.com/Game-of-thrones-the-ghost-of-harrenhal-ann

In [9]:
dat = pd.DataFrame(episodes)
dat.columns = ["season_idx", "episode_idx", "character", "utterance"]
dat.to_csv("got.csv")

In [6]:
pd.options.display.max_colwidth = 1000
dat

NameError: name 'dat' is not defined

In [16]:
for char in set(dat['character']):
    print (char)

RANGER
SER VANCE
SHIREEN
AERSON
STYR
CLIENT
CUT TO 
SANDOR
TOGETHER
DAVOS
FREY SOLDIER
CAPTAIN
GOLD CLOAK
EDD
SEPTA
BROTHERS
OTHELL YARWICK
AXELL FLORENT
ALLISER
GARED
WALDA
LOLLYS
YOREN
HOT PIE
AYRA
LYSA
MANCE
 MISSANDEI
CUT  TO
TORTURER
YOUNG RODRIK
MOSSADOR
CATELYIN
MUSICIAN
PROTESTER
ELARIA
S WATCHMAN
FIRST MATE
ROOSE
EDDISION
SAND SNAKES
MANSERVANT
SER JORAH
TRYSTANE
SAMWELL
BRAND
TYRIOM
YEZZAN
LITTLEFINGER
HEAD PROSTITUTE
EVERYONE
THE HOUND
RENLY DWARF
BRON
REGINALD
STRONG
LANCEL
MOSADOR
MYRCELLA
ORELL
THE OTHERS AT THE TABLE
JOFFREY
DOTHRAKI WOMAN
WAYMAR ROYCE
MELESSA
ELDER MEEREEN SLAVE
AHSA
BARRISTAN
MASTER OF ARMS
GERARD
QYBURN
BRINENNE
DOLROUS EDD
LORD MORMONT
LADY OLENNA
WALDER
JAQEN
PORTAN
PYELLE
YTRION
BIANCA
BERIC
HANDMAIDEN
KNIGHT
DOREAH
PYATT PREE
RANDYLL
PRIESTESS
A VOICE
LORD KARSTARK
LORD ROYCE
DAUGHTER
DROGO
SAMMY
WILDLING ELDER
MAGGY
RENLY
RIDER
MYRANDA
RODRIK
ALL TOGETHER
MAESTER
ROSLIN
KINVARA
VISERYS
IRRI
RAMSAY
SALLADOR
WATCHMAN
SHADOW TOWER BROTHER
WILDLING
T

# PROCESSING

In [11]:
import pandas as pd
import numpy as np

In [15]:
dat = pd.read_csv('got.csv')

In [25]:
def getCharacterLines(aliases):
    lines = np.array( [] )
    for alias in aliases:
        newlines = dat[dat['character'] == alias]['utterance']
        lines = np.append(lines, newlines)
    return lines
#dat[dat['character'] == 'JON SNOW'] + dat[dat['character'] == 'JON SNOW']

In [26]:
JONSNOW = getCharacterLines(['JON SNOW', 'JON'])
characterLines = {'JONSNOW' : JONSNOW}

In [27]:
JONSNOW.shape

(624,)

In [28]:
for line in JONSNOW:
    print(line)

Mance was a ranger.
When I'm free, will I be free to go?
What happened to their fathers?
My father told me big men fall just as quick as little ones if you put a sword through their hearts.
Jon Snow. JON kneels before TORMUND, mistaking him for MANCE RAYDER, the King-beyond-the-Wall.
Your Grace. The wildlings laugh
The Lord Commander sent me to the Halfhand for seasoning.
He wants me to lead one day.
If I'm a traitor, then you are, too. TORMUND glowers.
(Hesitates.) I want to be free.
We stopped at Craster's Keep on the way north. I saw...
I saw Craster take his own baby boy and leave it in the woods. I saw what took it.
Because when I told the Lord Commander, he already knew. Thousands of years ago, the First Men battled the White Walkers and defeated them. I want to fight for the side that fights for the living. Did I come to the right place?
They stabbed me . Olly...he put a knife in my heart . I shouldn’t be here .
Nothing . There was nothing at all .
I did what I thought was right